In [3]:
from pathlib import Path
import pandas as pd
from skimage.feature import peak_local_max
import mrcfile
import napari
import starfile

In [8]:
eman2_dir = Path("/media/michalakdj/scratch/eman2_models/")
tomo_paths = [
    eman2_dir / Path("tomograms/ts001_corrected.mrc"),
    eman2_dir / Path("tomograms/ts002_corrected.mrc"),
    eman2_dir / Path("tomograms/ts003_corrected.mrc"),
    eman2_dir / Path("tomograms/ts004_corrected.mrc"),
    eman2_dir / Path("tomograms/ts005_corrected.mrc"),
]
tomo_names = [tomo_path.stem.split('_')[0] for tomo_path in tomo_paths]
segment_paths = [
    eman2_dir / Path("segmentations/ts001_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts002_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts003_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts004_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts005_corrected__ribosome_good_seg.mrc"),
]
tomo_bin_factor = 10
peak_separation = 12 # pixels
unbinned_pixel_size = 1.0825
print(f"Peaks will be at least {peak_separation * tomo_bin_factor * unbinned_pixel_size / 10.0} nm apart.")
threshold_rel = 0.1
viewer = napari.Viewer(ndisplay=3)

Peaks will be at least 12.99 nm apart.


In [9]:
# View the tomogram in Napari
tomos = [mrcfile.read(tomo) for tomo in tomo_paths]
layers = {}
name_i = 0
for tomo in tomos:
    layers[f"{tomo_names[name_i]}"] = viewer.add_image(
        tomo, 
        name=f"{tomo_names[name_i]}",
        depiction="plane", 
        blending="translucent", 
        visible=False,
        )
    name_i+=1

In [6]:
# Add the segmentation to the Napari viewer
segment_maps = [mrcfile.read(segment_path) for segment_path in segment_paths]

name_i = 0
for segment_map in segment_maps:
    layers[f"{tomo_names[name_i]}_seg"] = viewer.add_image(
        segment_map, 
        name=f"{tomo_names[name_i]}_seg",
        colormap="gist_earth", 
        blending="translucent",
        visible=False, 
        )
    name_i+=1

In [7]:
colors = ["red","orange", "yellow","green", "blue", "purple"]

name_i = 0
peak_coords_dict = {}
for segment_map in segment_maps:
    peak_coords_dict[tomo_names[name_i]] = peak_local_max(
        segment_map, 
        min_distance=10, 
        threshold_rel=threshold_rel, 
        ) 
    layers[f"{tomo_names[name_i]}_peaks"] = viewer.add_points(
        peak_coords_dict[tomo_names[name_i]], 
        name=f"{tomo_names[name_i]} Peaks(rel_density > {threshold_rel})", 
        edge_color=colors[0], 
        edge_width=0.2, 
        blending="translucent",
        visible=False, 
        )
    name_i+=1

In [ ]:
layers["ts001"].bounding_box.visible = True

In [ ]:
ribosome = mrcfile.read("/media/michalakdj/scratch/ribosome/Refine3D/clean1_bin8/run_class001.mrc")
ribosome_layer = viewer.add_image(ribosome)
ribosome_layer.bounding_box.visible = True

In [ ]:
# Convert the calculated peak coordinates from binned space to unbinned space. Star files for RELION required unbinned coordinates.
rln_coordinates = ["rlnCoordinateZ", "rlnCoordinateY", "rlnCoordinateX"]
rln_angles = ["rlnAngleRot", "rlnAngleTilt", "rlnAnglePsi"]

#rln_tomoname = "ts002"
#peak_coords_df = pd.DataFrame(peak_coords,columns=rln_coordinates)
#unbinned_peak_coords_df = pd.DataFrame()
#for coordinate in rln_coordinates:
#    unbinned_peak_coords_df[coordinate] = tomo_bin_factor * peak_coords_df[coordinate]
#for angle in rln_angles:
#    unbinned_peak_coords_df[angle] = 0.0
#unbinned_peak_coords_df["rlnTomoName"] = rln_tomoname
#unbinned_peak_coords_df.head(3)
#
# Create empty pd columns and append entries for every tomogram in tomos
name_i = 0
unbinned_peak_coords_dict = {}

import numpy as np

for tomo in peak_coords_dict:
    peak_coords_dict[tomo]= pd.DataFrame(peak_coords_dict[tomo], columns=rln_coordinates)
    unbinned_peak_coords_dict[tomo] = peak_coords_dict[tomo] * tomo_bin_factor
    for angle in rln_angles:
        random_angles = np.random.randint(0,179,size=len(unbinned_peak_coords_dict[tomo].index))
        unbinned_peak_coords_dict[tomo][angle] = random_angles
    unbinned_peak_coords_dict[tomo]["rlnTomoName"] = tomo_names[name_i]
    name_i+=1

In [4]:
write_path = Path("/media/michalakdj/scratch/eman2_models/segmentations/ribosomes.star")
#starfile.write(unbinned_peak_coords_df, write_path, overwrite=True)

In [5]:
write_path.name

'ribosomes.star'